In [ ]:
import os
import pandas as pd 
import numpy as np
from scipy.stats import rankdata
from scipy.stats import f
import matplotlib.pyplot as plt
import seaborn as sns
import time
import math

### Orange necessary code has been copy-pasted from github source (library issues)

In [ ]:
import orange ##file orange.py

## Loading data

In [ ]:
all_models_elec_h1 = pd.read_csv("all_models_elec_h1.csv")
all_models_elec_h2 = pd.read_csv("all_models_elec_h2.csv")
all_models_exch_h1 = pd.read_csv("all_models_exch_h1.csv")
all_models_exch_h2 = pd.read_csv("all_models_exch_h2.csv")

## Functions

In [ ]:
def select_topNworse_for_each_model(dataframe): 
    
    num_to_pick = 1
    
    nrows = dataframe.shape[0]
    ncols = dataframe.shape[1]
    ranking = pd.DataFrame(dataframe)
    for i in range(nrows):
        ranking.iloc[i,:] = rankdata(dataframe.iloc[i,:])
    avranks = [0]*ncols
    for i in range(ncols):
        avranks[i] = ranking.iloc[:,i].mean() 
    index_sorted = pd.DataFrame(avranks).sort_values(0).index
    top_columns = dataframe.columns[index_sorted]
    

    frameworks = ["autogluon","autosklearn","h2o","tpot"]
    cs = top_columns
    res_0 = [i for i in cs if frameworks[0] in i]
    res_1 = [i for i in cs if frameworks[1] in i]
    res_2 = [i for i in cs if frameworks[2] in i]
    res_3 = [i for i in cs if frameworks[3] in i]
    rest =  [i for i in cs if (frameworks[0] not in i) and (frameworks[1] not in i) and (frameworks[2] not in i) and (frameworks[3] not in i)]
    
    
    a = res_0[:1] 
    a.extend(res_0[-1:])
    b = res_1[:1]
    b.extend(res_1[-1:])
    c = res_2[:1]
    c.extend(res_2[-1:])
    d = res_3[:1]
    d.extend(res_3[-1:])
    
    rest.extend(a)
    rest.extend(b)
    rest.extend(c)
    rest.extend(d)
    
    return dataframe[rest]

In [ ]:
def select_top_20(dataframe):
    nrows = dataframe.shape[0]
    ncols = dataframe.shape[1]
    ranking = pd.DataFrame(dataframe)
    for i in range(nrows):
        ranking.iloc[i,:] = rankdata(dataframe.iloc[i,:])
    avranks = [0]*ncols
    for i in range(ncols):
        avranks[i] = ranking.iloc[:,i].mean() 
    index_sorted = pd.DataFrame(avranks).sort_values(0).index
    top_columns = dataframe.columns[index_sorted]
    top20 = top_columns[:20]

    return dataframe[top20]

## Function to perform test

In [ ]:
def perform_friedman_nemenyi_test(filename, dataframe, alpha = 0.05):
    
    nrows = dataframe.shape[0]
    ncols = dataframe.shape[1]
    
    #perform ranking of the results
    ranking = pd.DataFrame(dataframe)
    for i in range(nrows):
        ranking.iloc[i,:] = rankdata(dataframe.iloc[i,:])
    
    #compute average of rankings
    avranks = [0]*ncols
    for i in range(ncols):
        avranks[i] = ranking.iloc[:,i].mean() 
    
    #compute Friedman and Iman tests, with respective degrees of freedom
    (f_stat, dof), (im, fdistdof) = compute_friedman(avranks, nrows)

    df1 = fdistdof[0]
    df2 = fdistdof[1]

    critical_value = f.ppf(q = 1 - alpha, dfn = df1, dfd = df2)

    if (im > critical_value):
        print("H0 Rejected, proceeding")
        
        cd = compute_CD(avranks, nrows, str(alpha))
        path = filename+"_CD.png"
        graph_ranks(avranks, dataframe.columns, cd=cd, width=6, textspace=1, filename=path, bbox_inches="tight")
        
    else:
        print("Cannot reject H0, stop.")

## Execution

## Electricity - h = 1 

In [ ]:
perform_friedman_nemenyi_test("Elec - H1",select_topNworse_for_each_model(all_models_elec_h1))

## Electricity - h = 2

In [ ]:
perform_friedman_nemenyi_test("Elec - H2",select_topNworse_for_each_model(all_models_elec_h2))

## Exchange - h = 1 

In [ ]:
perform_friedman_nemenyi_test("Exch - H1",select_topNworse_for_each_model(all_models_exch_h1))

## Exchange - h = 2

In [ ]:
perform_friedman_nemenyi_test("Exch - H2",select_topNworse_for_each_model(all_models_exch_h2))

In [ ]:
elec_all = pd.concat([all_models_elec_h1, all_models_elec_h2],axis=0)
elec_all = elec_all.groupby(level=0).mean()
exch_all = pd.concat([all_models_exch_h1, all_models_exch_h2],axis=0)
exch_all = exch_all.groupby(level=0).mean()

## Electricity - Both horizons

In [ ]:
perform_friedman_nemenyi_test("Electricity",select_topNworse_for_each_model(elec_all))

## Exchange - Both horizons

In [ ]:
perform_friedman_nemenyi_test("Exchange",select_topNworse_for_each_model(exch_all))

In [ ]:
both_all = pd.concat([elec_all, exch_all],axis=0)
both_all = both_all.groupby(level=0).mean()

## All data together

In [ ]:
perform_friedman_nemenyi_test("Both",select_topNworse_for_each_model(both_all))

In [ ]:
perform_friedman_nemenyi_test("Both",select_top_20(both_all))